<a href="https://colab.research.google.com/github/sathyanarayanajammala/GenAI/blob/main/CTGAN_Insurance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In this tutorial we use the defaul version of the CTGAN:
https://github.com/sdv-dev/CTGAN

We also need to install table evaluator to evaluate the quality of the generated synthetic data.

In [2]:
%%capture
!pip install ctgan
!pip install table_evaluator
!pip install kaggle
!pip install kagglehub

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mirichoi0218/insurance")

print("Path to dataset files:", path)

100%|██████████| 16.0k/16.0k [00:00<00:00, 31.2MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/mirichoi0218/insurance/versions/1


Now we download a dataset that includes both continuous and discrete values. I chose a random dataset from Kaggle: https://www.kaggle.com/datasets/mirichoi0218/insurance

In [4]:
import pandas as pd
data = pd.read_csv('/root/.cache/kagglehub/datasets/mirichoi0218/insurance/versions/1/insurance.csv')

In [5]:
data

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


Next, we define a list with column names for categorical variables. This list will be passed to the model so that the model can decide how to process these fields.

In [6]:
categorical_features = ['age','sex','children','smoker','region']

#Model training

Next, we simply define an instance of CTGANSynthesizer and call the fit method with the dataframe and the list of categorical variables.

We train the model for 300 epochs only as the discriminator and generator loss becomes quite low after these many epochs.

In [7]:
from ctgan import CTGAN

ctgan = CTGAN(verbose=True)
ctgan.fit(data, categorical_features, epochs = 200)

Gen. (0.00) | Discrim. (0.00):   0%|          | 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:825: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Gen. (0.18) | Discrim. (0.03): 100%|██████████| 200/200 [00:12<00:00, 15.73it/s]


#Synthetic data generation

In [8]:
samples = ctgan.sample(1000)

In [9]:
samples

,age,sex,bmi,children,smoker,region,charges
0,38,female,31.362583,3,no,southeast,23702.652886
1,42,male,20.502155,0,no,southwest,18806.376411
2,33,female,47.967894,0,no,northeast,16136.502166
3,50,female,21.607502,1,no,northeast,18307.007611
4,45,female,25.442154,1,no,northeast,61080.031694
...,...,...,...,...,...,...,...
995,30,female,29.905324,2,no,southeast,18126.233696
996,32,female,24.963159,0,yes,northwest,31541.171522
997,40,female,32.674181,1,no,northwest,14776.145062
998,57,female,21.525743,0,no,southwest,3033.072813


In [11]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=samples)

https://docs.google.com/spreadsheets/d/1wSbK6Pg_IFHhnWew_Nmo1jjveXSDpvYtpHhQXZlBJ70#gid=0


#Evaluation